In [9]:
import sys
import pandas as pd
from itertools import combinations
import numpy as np

sys.path.insert(0,'../')
from core import core
from src import Option

In [10]:
Call_list=None
if Call_list:
    print("Call_list exists")
else:
    # ! if run .ipynb, input should be -> "../conf/config.json"
    Call_list = core.load_option(config_path="../conf/config.json")

In [11]:
def find_contracts_with_same_id(contracts):
    """
    找出具有相同合约编号前缀和合约类型的合约，并返回一个字典，其中键是合约编号前缀，值是另一个字典，
    其中键是合约类型( Call 或 Put )，值是具有相同前缀和类型的合约列表。

    Args:
    contracts (list): 合约列表。

    Returns:
    dict: 键是合约编号前缀，值是另一个字典，其键是合约类型，值是具有相同前缀和类型的合约列表。
    """
    contracts_dict = {}
    for option in contracts:
        contract_code = option.contract_code
        id_prefix, option_type, _ = contract_code.split('-')
        if id_prefix not in contracts_dict:
            contracts_dict[id_prefix] = {'C': [], 'P': []}
        contracts_dict[id_prefix][option_type].append(option)
    return contracts_dict


In [12]:
# 查找具有相同合约编号前缀的合约
contracts_dict = find_contracts_with_same_id(Call_list)

# 输出结果
print(f"数据集中共有 {len(contracts_dict)} 个合约\n")

for id_prefix, same_contracts in contracts_dict.items():
    call_contracts = same_contracts['C']
    put_contracts = same_contracts['P']
    
    n_call = len(call_contracts)
    n_put = len(put_contracts)
    print(f"每个 {id_prefix} 合约有 {n_call} 份 Call, {n_put} 份 Put")
    
    if id_prefix=="IO2105":
        break 



数据集中共有 92 个合约

每个 IO2101 合约有 29 份 Call, 29 份 Put
每个 IO2102 合约有 27 份 Call, 27 份 Put
每个 IO2103 合约有 40 份 Call, 40 份 Put
每个 IO2104 合约有 27 份 Call, 27 份 Put
每个 IO2105 合约有 27 份 Call, 27 份 Put


In [13]:
def pair_elements(lst):
    pairs = list(combinations(lst, 2))
    return pairs

def align_contract(input_option,operation):
    df = pd.DataFrame.from_dict(input_option._args())
    option_df=pd.DataFrame()

    for index in range(len(df)):
        new_option = Option.Option(
            contract_code=df.iloc[index]['contract_code'],
            option_price=df.iloc[index]['option_price'],
            underlying_price=df.iloc[index]['underlying_price'],
            strike_price=df.iloc[index]['strike_price'],
            risk_free_rate=0.02,
            operation=operation,
            volatility=df.iloc[index]['volatility'],
            implied_volatility=None,
            date=df.iloc[index]['date'].strftime('%Y-%m-%d'),
            Time_to_maturity=float((df.iloc[index]['end_date'] - df.iloc[index]['date']).days),
            tau=float((df.iloc[index]['end_date'] - df.iloc[index]['date']).days / 365.0),
        )
        new_option._calculate_black_scholes_price()
        new_option.calculate_greeks()
        option_df = pd.concat([option_df, pd.DataFrame([new_option.__dict__])], ignore_index=True)
        option_price = new_option.option_price
        
        a = 1 if operation == 'buy' else -1 if operation == 'sell' else None
        if a is None:
            raise ValueError('operation should be buy or sell')
        option_df['bs_expected_return'] =  a * (new_option.calculate_price() - option_price)
        option_df['volatility'] = new_option.implied_volatility

    option_df['exect_return'] = a * (df['option_price'] - df['option_price'].iloc[-1])   
        
    return option_df


In [14]:
def merge_data(long_df, short_df, selected_columns):
    def subtract_columns(df, suffixes):
        for suffix in suffixes:
            cols = [f'{suffix}_x', f'{suffix}_y']
            df[suffix] = df[cols[0]] - df[cols[1]]
        df.drop([f'{suffix}_x' for suffix in suffixes] + [f'{suffix}_y' for suffix in suffixes], axis=1, inplace=True)
   
    
    def sum_columns(df, suffixes):
        for suffix in suffixes:
            cols = [f'{suffix}_x', f'{suffix}_y']
            df[suffix] = df[cols].sum(axis=1)
        df.drop([f'{suffix}_x' for suffix in suffixes] + [f'{suffix}_y' for suffix in suffixes], axis=1, inplace=True)
    
    long_df = long_df[selected_columns]
    short_df = short_df[selected_columns]
    merged_df = pd.merge(long_df, short_df, on='date', how='outer')
    
    # 合并 contract_code
    merged_df['contract_code'] = merged_df['contract_code_x'] + '-' + merged_df['contract_code_y'].str.split('-').str[-1]
    
    # 合并希腊字母
    sub_suffixes = ['DELTA', 'THETA', 'VEGA', 'RHO']
    sum_suffixes = ['GAMMA', 'bs_expected_return', 'exect_return']
    sum_columns(merged_df, sum_suffixes)      
    subtract_columns(merged_df, sub_suffixes)
    # 使用均值合并波动率
    merged_df['volatility'] = merged_df[['volatility_x', 'volatility_y']].mean(axis=1)

    # 删除带有后缀的列
    merged_df.drop(['contract_code_x', 'volatility_x', 'contract_code_y', 'volatility_y'], axis=1, inplace=True)
    
    # 重新排序列
    merged_df = merged_df[selected_columns]
    merged_df = merged_df.sort_values(by='date')
    return merged_df
    
def bull_spread(contract_pair, id_prefix):
    """
    牛市看涨期权套利：买进一个执行价格较低的看涨期权, 同时卖出一个到期日相同、但执行价格较高的看涨期权，以利用两种期权之间的价差波动寻求获利，要求先付权利金，更适合波动率低的情况。

    牛市看跌期权套利：买进一个执行价格较低的看跌期权, 同时operation,output_option期日相同、但执行价格较高的看跌期权, 可以先收权利金, 适合波动率高的情况。
    """

    long_contract = contract_pair[0]
    long_contract.operation = "buy"
    short_contract = contract_pair[1]
    short_contract.operation = "sell"
    
    if "-C-" in long_contract.contract_code:
        stragety = (f"{id_prefix}-C-{long_contract.contract_code.split('-')[-1]}-{short_contract.contract_code.split('-')[-1]}")
    elif "-P-" in long_contract.contract_code:
        stragety = (f"{id_prefix}-P-{long_contract.contract_code.split('-')[-1]}-{short_contract.contract_code.split('-')[-1]}")
    else:
        raise ValueError("Contracts type error")


    long_df=align_contract(long_contract,'buy')
    short_df=align_contract(short_contract,'sell')
    
    selected_columns = ['date', 'contract_code', 'volatility', 'DELTA', 'THETA', 'GAMMA', 'VEGA', 'RHO', 'bs_expected_return', 'exect_return']
    merged_df = merge_data(long_df, short_df,selected_columns)
    return merged_df, stragety

def bear_spread(option_pairs, id_prefix):
    pass

In [15]:
def save_stragety_pnl(PNL_df, stragety, type):
    filename = f"../db/result/strageties/{stragety}.xlsx"
    with pd.ExcelWriter(filename, engine='xlsxwriter') as writer:
        PNL_df.to_excel(writer, sheet_name=f'{stragety}', index=False)


def construct_spreads(contracts_dict):
    for id_prefix, same_contracts in contracts_dict.items():
        call_contracts = same_contracts['C']
        put_contracts = same_contracts['P']
        
        if len(call_contracts) > 0 and len(put_contracts) > 0:
            bull_PNL_df,bear_PNL_df=pd.DataFrame(),pd.DataFrame()
            
            # 生成所有可能的组合
            call_pairs = pair_elements(call_contracts)
            put_pairs = pair_elements(put_contracts)
            
            for contract_pair in call_pairs:
                bull_portfoio_df,stragety=bull_spread(contract_pair,id_prefix)
                bull_PNL_df=pd.concat([bull_portfoio_df],ignore_index=True)
                save_stragety_pnl(bull_PNL_df,stragety,'call')    
                            
                bear_portfoio_df,stragety=bull_spread(contract_pair,id_prefix)
                bear_PNL_df=pd.concat([bear_portfoio_df], ignore_index=True)
                save_stragety_pnl(bear_PNL_df,stragety,'call')
                
            for contract_pair in put_pairs:
                bull_portfoio_df,stragety=bull_spread(contract_pair,id_prefix)
                bull_PNL_df=pd.concat([bear_portfoio_df], ignore_index=True)
                save_stragety_pnl(bull_PNL_df,stragety,'put')
                
                bear_portfoio_df,stragety =bear_spread(contract_pair,id_prefix)
                bear_PNL_df=pd.concat([bear_portfoio_df], ignore_index=True)
                save_stragety_pnl(bull_PNL_df,stragety,'put')

        else:
            raise ValueError("No contracts available")

In [16]:
construct_spreads(contracts_dict)


x:\jacky\homework\deravative hedge\hedge\bin\..\src\Option.py:142: RuntimeWarning: divide by zero encountered in scalar divide
  d_1 = (log(S / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * sqrt(T))
x:\jacky\homework\deravative hedge\hedge\bin\..\src\Option.py:159: RuntimeWarning: divide by zero encountered in scalar divide
  d1 = (log(S / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * sqrt(T))
x:\jacky\homework\deravative hedge\hedge\bin\..\src\Option.py:244: RuntimeWarning: divide by zero encountered in scalar divide
  d1 = (log(S / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * sqrt(T))
x:\jacky\homework\deravative hedge\hedge\bin\..\src\Option.py:249: RuntimeWarning: invalid value encountered in scalar divide
  self.THETA = -((S * norm.pdf(d1) * sigma) / (2 * sqrt(T))) - (r * K * exp(-r * T) * norm.cdf(d2))
x:\jacky\homework\deravative hedge\hedge\bin\..\src\Option.py:250: RuntimeWarning: invalid value encountered in scalar divide
  self.GAMMA = norm.pdf(d1) / (S * sigma * sqrt(T))
x:\jack